In [1]:
# In the first Python interactive shell
from os import cpu_count
import numpy as np
a = np.random.random(int(1e6)-1)*100  # Start with an existing NumPy array

from multiprocessing import shared_memory, Process
shm = shared_memory.SharedMemory(create=True, size=a.nbytes)
# Now create a NumPy array backed by shared memory
b = np.ndarray(a.shape, dtype=a.dtype, buffer=shm.buf)
b[:] = a[:]  # Copy the original data into shared memory
#b

In [2]:
shm.name
b

array([92.85026762, 25.78493521, 18.58754435, ..., 17.50811649,
       62.76084201, 47.64986271])

In [3]:
def divide_work(ARRAY):
    n_cpus = cpu_count()
    job_len = np.floor(ARRAY.shape[0]/n_cpus)
    work_schedule = np.zeros((n_cpus,2), dtype=np.longlong) # [(start_position),(job_length)]
    for i in range(0,n_cpus):
        distance = i*job_len
        work_schedule[i,0] =  distance 
        if i != (n_cpus - 1): # The length of the last job is dynamic 
            work_schedule[i,1] = job_len
        else:
            work_schedule[i,1] = ARRAY.shape[0] - distance
    return work_schedule

In [4]:
job_list = divide_work(b)
job_list

array([[     0, 249999],
       [249999, 249999],
       [499998, 249999],
       [749997, 250002]], dtype=int64)

In [6]:
def is_even(ARRAY,OUTPUT):
    for i in range(len(ARRAY)):   
        if (np.round(ARRAY[i]) % 2) == 0.0:
            OUTPUT[i] = 1
        else:
            OUTPUT[i] = 0

def assign_work(INPUT_ARRAY, TASK, WORK_SCHEDULE, OUTPUT):
    processes = []
    for i in range(len(WORK_SCHEDULE)):
        print(f'start {i}')
        position = WORK_SCHEDULE[i,0]
        magnitude = WORK_SCHEDULE[i,1]
        jobslice = INPUT_ARRAY[position:int(position+magnitude)]
        jobslice_str = str(jobslice)
        outslice = OUTPUT[position:int(position+magnitude)]
        print(jobslice_str)
        p = Process(target=is_even, args=(jobslice,outslice))
        processes.append(p)
        p.start()
    for i in range(len(WORK_SCHEDULE)):
        print(f'join {i}')
        processes[i].join()

In [7]:
if __name__ == '__main__':
    assign_work(a,is_even,job_list,b)

start 0
[92.85026762 25.78493521 18.58754435 ... 97.69429102 49.57606766
 76.58180629]
start 1
[57.98520219 47.0363109  46.14228652 ... 34.55737148 56.28191419
 16.37358923]
start 2
[29.88175353 85.65194254 41.39741925 ... 59.29121039 48.84750195
 10.09974622]
start 3
[84.75808106 95.44707419  2.04180107 ... 17.50811649 62.76084201
 47.64986271]
join 0
join 1
join 2
join 3


In [20]:
print(len(b))
b

999999


array([1., 1., 1., ..., 1., 1., 0.])